In [1]:
import pandas as pd
from datetime import datetime, timedelta
import ast

# --- Configuration Constants ---
# The maximum stock level for each product after a restock.
MAX_STOCK = 1000
# The day of the month when restocking occurs.
RESTOCK_DAY = 5
# The name of the input CSV file.
CSV_FILE_PATH = '10000.csv'

In [3]:
import pandas as pd
import ast
from datetime import datetime, timedelta

# Constants
CSV_FILE_PATH = "bills.csv"
MAX_STOCK = 100
RESTOCK_DAY = 5


def load_and_preprocess_data(file_path):
    """
    Load and preprocess bill data from CSV file.
    
    Args:
        file_path (str): Path to the CSV file
        
    Returns:
        pd.DataFrame: Preprocessed transaction data
    """
    try:
        # Load CSV with datetime parsing
        df = pd.read_csv(file_path, parse_dates=['Date_Time'])
    except FileNotFoundError:
        print(f"Error: File '{file_path}' not found.")
        return pd.DataFrame()
    except Exception as e:
        print(f"Error loading file: {e}")
        return pd.DataFrame()

    # Columns containing string representations of lists
    list_columns = ['Product_IDs', 'Quantities', 'Products_With_Brands']

    # Convert string lists to actual lists
    for col in list_columns:
        df[col] = df[col].apply(ast.literal_eval)

    # Explode lists to create one row per item
    df_exploded = df.explode(list_columns).reset_index(drop=True)

    # Rename columns for clarity
    df_exploded.rename(columns={
        'Product_IDs': 'Product_ID',
        'Quantities': 'Quantity',
        'Products_With_Brands': 'Product_Name'
    }, inplace=True)

    # Convert quantity to numeric
    df_exploded['Quantity'] = pd.to_numeric(df_exploded['Quantity'])

    # Sort by date and time
    df_exploded.sort_values('Date_Time', inplace=True)
    df_exploded.reset_index(drop=True, inplace=True)

    return df_exploded


def simulate_inventory(transactions_df, target_date, max_stock):
    """
    Simulate inventory levels up to target date.
    
    Args:
        transactions_df (pd.DataFrame): Transaction data
        target_date (datetime): Date to calculate stock for
        max_stock (int): Maximum stock level
        
    Returns:
        tuple: (previous_stock, current_stock, product_names)
    """
    if transactions_df.empty:
        return {}, {}, {}

    # Get all unique products
    all_products = transactions_df['Product_ID'].unique()
    
    # Create product name mapping
    product_names = (transactions_df.drop_duplicates('Product_ID')
                    .set_index('Product_ID')['Product_Name'].to_dict())

    # Initialize stock levels
    stock = {pid: max_stock for pid in all_products}

    # Start from first day of month containing first transaction
    first_date = transactions_df['Date_Time'].min().date()
    start_date = first_date.replace(day=1)

    # Simulate day by day
    current_date = start_date
    previous_stock = None
    
    while current_date <= target_date.date():
        # Restock on the 5th of each month
        if current_date.day == RESTOCK_DAY:
            stock = {pid: max_stock for pid in all_products}

        # Capture stock at start of target date
        if current_date == target_date.date():
            previous_stock = stock.copy()

        # Process daily sales
        daily_sales = transactions_df[transactions_df['Date_Time'].dt.date == current_date]
        for _, sale in daily_sales.iterrows():
            stock[sale['Product_ID']] -= sale['Quantity']

        current_date += timedelta(days=1)

    current_stock = stock
    
    return previous_stock, current_stock, product_names


def generate_report(target_date, previous_stock, current_stock, product_names):
    """
    Generate and print inventory report.
    
    Args:
        target_date (datetime): Target date
        previous_stock (dict): Stock at start of day
        current_stock (dict): Stock at end of day
        product_names (dict): Product ID to name mapping
    """
    if not previous_stock:
        print("No data available for the given date.")
        return

    print("\n" + "="*90)
    print(f"Inventory Stock Report for: {target_date.strftime('%Y-%m-%d')}")
    print("="*90)
    print(f"{'Product ID':<15} {'Product Name':<45} {'Previous Stock':>15} {'Current Stock':>15}")
    print("-"*90)

    # Sort products for consistent output
    for pid in sorted(previous_stock.keys()):
        name = product_names.get(pid, "Unknown")
        prev_stock = previous_stock[pid]
        curr_stock = current_stock[pid]
        print(f"{pid:<15} {name:<45} {prev_stock:>15} {curr_stock:>15}")
    
    print("="*90)


def main():
    """Main function to run inventory simulation."""
    print("--- Inventory Stock Simulation ---")
    
    # Load data
    transactions = load_and_preprocess_data(CSV_FILE_PATH)
    if transactions.empty:
        return

    # Get target date from user
    while True:
        try:
            date_input = input("Enter target date (YYYY-MM-DD): ")
            target_date = datetime.strptime(date_input, "%Y-%m-%d")
            break
        except ValueError:
            print("Invalid format. Please use YYYY-MM-DD.")

    # Run simulation
    prev_stock, curr_stock, names = simulate_inventory(transactions, target_date, MAX_STOCK)

    # Generate report
    generate_report(target_date, prev_stock, curr_stock, names)
if __name__ == "__main__":
    main()

--- Inventory Stock Simulation ---

Inventory Stock Report for: 2025-03-28
Product ID      Product Name                                   Previous Stock   Current Stock
------------------------------------------------------------------------------------------
PVSL0101011125  Basmati Rice (Daawat)                                    -368            -382
PVSL0101021125  Basmati Rice (India Gate)                                -570            -640
PVSL0101031125  Basmati Rice (Kohinoor)                                  -443            -473
PVSL0101041125  Basmati Rice (Tilda)                                     -385            -402
PVSL0101051125  Basmati Rice (Fortune)                                   -396            -410
PVSL0102010126  Sona Masoori Rice (Sri Lalitha)                          -444            -464
PVSL0102020126  Sona Masoori Rice (Pride of India)                       -367            -393
PVSL0102030126  Sona Masoori Rice (Fortune)                              -458     